In [2]:
import requests
import pandas as pd 
import numpy as np
import re
import sys, getopt
import csv
import os
import matplotlib.pyplot as plt
import seaborn as sns

def load_csv_dataset(file_path):
    return pd.read_csv(file_path)

In [3]:
os.chdir('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings')
os.getcwd()

'/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings'

In [4]:
latest_match_data = load_csv_dataset('create_dataset_1/CSVs we use often/newest_edited_match_dataset_may_30.csv')
pd.set_option('display.max_columns', None)

## Quick dataset cleaning

In [5]:
latest_match_data.loc[latest_match_data['Team.1.Name'] == 'Seychelles', 'Team.1.Code'] = 'SCH'
latest_match_data.loc[latest_match_data['Team.2.Name'] == 'Seychelles', 'Team.2.Code'] = 'SCH'

latest_match_data.loc[latest_match_data['Team.1.Name'] == 'Nicaragua', 'Team.1.Code'] = 'NIC'
latest_match_data.loc[latest_match_data['Team.2.Name'] == 'Nicaragua', 'Team.2.Code'] = 'NIC'

latest_match_data.loc[latest_match_data['Team.1.Name'] == 'Democratic Republic of the Congo', 'Team.1.Name'] = 'DR Congo'
latest_match_data.loc[latest_match_data['Team.2.Name'] == 'Democratic Republic of the Congo', 'Team.2.Name'] = 'DR Congo'

latest_match_data.loc[latest_match_data['Team.1.Name'] == 'Swaziland', 'Team.1.Name'] = 'Eswatini'
latest_match_data.loc[latest_match_data['Team.2.Name'] == 'Swaziland', 'Team.2.Name'] = 'Eswatini'

In [6]:
len(latest_match_data['Team.1.Code'].unique()), len(latest_match_data['Team.1.Name'].unique())

# Group by 'Team.1.Code' and filter groups having more than one unique 'Team.1.Name'
grouped = latest_match_data.groupby('Team.1.Code').filter(lambda x: x['Team.1.Name'].nunique() > 1)

# Display the rows where 'Team.1.Code' has multiple 'Team.1.Name'
print(grouped[['Team.1.Code', 'Team.1.Name']].drop_duplicates())

Empty DataFrame
Columns: [Team.1.Code, Team.1.Name]
Index: []


In [7]:
len(latest_match_data['Team.2.Code'].unique()), len(latest_match_data['Team.2.Name'].unique())

# Group by 'Team.1.Code' and filter groups having more than one unique 'Team.1.Name'
grouped = latest_match_data.groupby('Team.2.Code').filter(lambda x: x['Team.2.Name'].nunique() > 1)

# Display the rows where 'Team.1.Code' has multiple 'Team.1.Name'
print(grouped[['Team.2.Code', 'Team.2.Name']].drop_duplicates())

Empty DataFrame
Columns: [Team.2.Code, Team.2.Name]
Index: []


## Task 1: does each team's win rate get higher the larger their MV difference

### Script Version

In [ ]:
# Assuming latest_match_data is your DataFrame
team1_unique = latest_match_data['Team.1.Name'].unique()
team2_unique = latest_match_data['Team.2.Name'].unique()

# Create the union of both unique lists
all_teams_unique = np.union1d(team1_unique, team2_unique)

In [ ]:
overall_table = pd.DataFrame()
for team in all_teams_unique:
    this_team_table = generate_win_rate_intervals_table_team(team, latest_match_data)
    overall_table = pd.concat([overall_table, this_team_table], ignore_index=True)


In [ ]:
overall_table.to_csv('win_rate_by_MV_interval_by_country.csv', index=False)

In [ ]:
def generate_win_rate_intervals_table_team(first_team, latest_match_data):

    #####PAGE 1#####
    #first_team = 'Albania'
    # Filter the rows where the team is either in 'Team.1.Name' or 'Team.2.Name'
    team_data = latest_match_data[(latest_match_data['Team.1.Name'] == first_team) | (latest_match_data['Team.2.Name'] == first_team)]

    # Create a new DataFrame to store the results
    columns = ['Country Name', 'Date', 'Competition', 'Score Difference', 'Market Value Difference', 'Team.1 Total Market Value', 'Team.2 Total Market Value', 'Season', 'Team Position']
    result_table = pd.DataFrame(columns=columns)

    # Populate the DataFrame
    for index, row in team_data.iterrows():
        if row['Team.1.Name'] == first_team:
            new_row = {
                'Country Name': first_team,
                'Date': row['Date'],
                'Competition': row['Competition'],
                'Score Difference': row['Score.Difference'],
                'Market Value Difference': row['Market.Value.Difference'],
                'Team.1 Total Market Value': row['Team.1.Total.Market.Value'],
                'Team.2 Total Market Value': row['Team.2.Total.Market.Value'],
                'Season': row['Season'],
                'Team Position': 'Team 1'
            }
        else:
            new_row = {
                'Country Name': first_team,
                'Date': row['Date'],
                'Competition': row['Competition'],
                'Score Difference': row['Score.Difference'],
                'Market Value Difference': row['Market.Value.Difference'],
                'Team.1 Total Market Value': row['Team.1.Total.Market.Value'],
                'Team.2 Total Market Value': row['Team.2.Total.Market.Value'],
                'Season': row['Season'],
                'Team Position': 'Team 2'
            }
        result_table = pd.concat([result_table, pd.DataFrame([new_row])], ignore_index=True)

    #####PAGE 2#####
    # Add new columns with default values
    result_table['Team Won?'] = False
    result_table['Team More Valuable?'] = False

    # Update the columns based on conditions
    for index, row in result_table.iterrows():
        if row['Team Position'] == 'Team 1':
            if row['Score Difference'] > 0:
                result_table.at[index, 'Team Won?'] = True
            if row['Market Value Difference'] > 0:
                result_table.at[index, 'Team More Valuable?'] = True
        elif row['Team Position'] == 'Team 2':
            if row['Score Difference'] < 0:
                result_table.at[index, 'Team Won?'] = True
            if row['Market Value Difference'] < 0:
                result_table.at[index, 'Team More Valuable?'] = True
    #####PAGE 3#####

    # Calculate the total number of matches
    less_valuable_matches = result_table[result_table['Team More Valuable?'] == False]
    more_valuable_matches = result_table[result_table['Team More Valuable?'] == True]
    if len(more_valuable_matches) > 0:
        more_valuable_matches['Value Difference (Abs)'] = abs(result_table['Market Value Difference'])
    else:
        0==0
    if len(less_valuable_matches) > 0:
        less_valuable_matches['Value Difference (Abs)'] = abs(result_table['Market Value Difference']) * -1

    #####PAGE 4#####
    #PRODUCING TABLE FROM MORE VALUABLE MATCHES
    interval_stats_ABOVE = pd.DataFrame(columns=['Team Name', 'Interval Range', 'Total Matches', 'Total Wins', 'Win Percentage'])
    if len(more_valuable_matches) == 0:
        0==0
    else:
        for i in range(5, 9):
            if (i == 7) or (i == 8) or (i == 6) or (i == 5):
                number_lower = 10 ** i
                number_higher = 10 ** (i + 1)
                #print('lower number: ', number_lower, i)
                #print('higher number: ', number_lower, i)
                interval_matches = more_valuable_matches[(more_valuable_matches['Value Difference (Abs)'] >= number_lower) & (more_valuable_matches['Value Difference (Abs)'] <= number_higher / 2)]
                num_matches = len(interval_matches)
                interval_wins = interval_matches[interval_matches['Team Won?'] == True]
                num_wins = len(interval_wins)
                win_percentage = (num_wins / num_matches) * 100 if num_matches > 0 else 0
                value_set = []
                for index, row in interval_matches.iterrows():
                    if row['Team Position'] == 'Team 1':
                        value_set.append(row['Team.1 Total Market Value'])
                    elif row['Team Position'] == 'Team 2':
                        value_set.append(row['Team.2 Total Market Value'])
                if value_set == []:
                    mean_mv_this_interval = 0
                    sd_mv_this_interval = 0
                    max_mv_this_interval = 0
                else:
                    mean_mv_this_interval = np.array(value_set).mean()
                    sd_mv_this_interval = np.array(value_set).std()
                    max_mv_this_interval = np.array(value_set).max()
                interval_row = {
                    'Team Name': first_team,
                    'Interval Range': f"${number_lower} - ${number_higher / 2}",
                    'Total Matches': num_matches,
                    'Total Wins': num_wins,
                    'Win Percentage': win_percentage,
                    'Mean MV, this interval': mean_mv_this_interval,
                    'SD MV, this interval': sd_mv_this_interval,
                    'Max MV, this interval': max_mv_this_interval
                }
                #print(i, 'interval 1: ', f"${number_lower} - ${number_higher / 2}")
                interval_stats_ABOVE = pd.concat([interval_stats_ABOVE, pd.DataFrame([interval_row])], ignore_index=True)

                interval_matches_ii = more_valuable_matches[(more_valuable_matches['Value Difference (Abs)'] >= (number_higher / 2)) & (more_valuable_matches['Value Difference (Abs)'] <= number_higher)]
                num_matches_ii = len(interval_matches_ii)
                interval_wins_ii = interval_matches_ii[interval_matches_ii['Team Won?'] == True]
                num_wins_ii = len(interval_wins_ii)
                win_percentage_ii = (num_wins_ii / num_matches_ii) * 100 if num_matches_ii > 0 else 0
                value_set_ii = []
                for index, row in interval_matches_ii.iterrows():
                    if row['Team Position'] == 'Team 1':
                        value_set_ii.append(row['Team.1 Total Market Value'])
                    elif row['Team Position'] == 'Team 2':
                        value_set_ii.append(row['Team.2 Total Market Value'])
                if value_set_ii == []:
                    mean_mv_this_interval = 0
                    sd_mv_this_interval = 0
                    max_mv_this_interval = 0
                else:
                    mean_mv_this_interval = np.array(value_set_ii).mean()
                    sd_mv_this_interval = np.array(value_set_ii).std()
                    max_mv_this_interval = np.array(value_set_ii).max()
                interval_row = {
                    'Team Name': first_team,
                    'Interval Range': f"${number_higher / 2} - ${number_higher}",
                    'Total Matches': num_matches_ii,
                    'Total Wins': num_wins_ii,
                    'Win Percentage': win_percentage_ii,
                    'Mean MV, this interval': mean_mv_this_interval,
                    'SD MV, this interval': sd_mv_this_interval,
                    'Max MV, this interval': max_mv_this_interval
                }
                #print(i, 'interval 2: ', f"${number_higher / 2} - ${number_higher}")
                interval_stats_ABOVE = pd.concat([interval_stats_ABOVE, pd.DataFrame([interval_row])], ignore_index=True)

            else:
                number_lower = 10 ** i
                number_higher = 10 ** (i + 1)
                interval_matches = more_valuable_matches[(more_valuable_matches['Value Difference (Abs)'] >= number_lower) & (more_valuable_matches['Value Difference (Abs)'] <= number_higher)]
                num_matches = len(interval_matches)
                interval_wins = interval_matches[interval_matches['Team Won?'] == True]
                num_wins = len(interval_wins)
                win_percentage = (num_wins / num_matches) * 100 if num_matches > 0 else 0
                value_set = []
                for index, row in interval_matches.iterrows():
                    if row['Team Position'] == 'Team 1':
                        value_set.append(row['Team.1 Total Market Value'])
                    elif row['Team Position'] == 'Team 2':
                        value_set.append(row['Team.2 Total Market Value'])
                if value_set == []:
                    mean_mv_this_interval = 0
                    sd_mv_this_interval = 0
                    max_mv_this_interval = 0
                else:
                    mean_mv_this_interval = np.array(value_set).mean()
                    sd_mv_this_interval = np.array(value_set).std()
                    max_mv_this_interval = np.array(value_set).max()
                interval_row = {
                    'Team Name': first_team,
                    'Interval Range': f"${number_lower} - ${number_higher / 2}",
                    'Total Matches': num_matches,
                    'Total Wins': num_wins,
                    'Win Percentage': win_percentage,
                    'Mean MV, this interval': mean_mv_this_interval,
                    'SD MV, this interval': sd_mv_this_interval,
                    'Max MV, this interval': max_mv_this_interval
                }
                interval_stats_ABOVE = pd.concat([interval_stats_ABOVE, pd.DataFrame([interval_row])], ignore_index=True)
        interval_stats_ABOVE['Interval Side'] = 'More Valuable'

    #####PAGE 5#####
    #PRODUCING TABLE FROM LESS VALUABLE MATCHES
    interval_stats_BELOW = pd.DataFrame(columns=['Team Name', 'Interval Range', 'Total Matches', 'Total Wins', 'Win Percentage'])
    if len(less_valuable_matches) == 0:
        0==0
    else:
        for i in range(5, 9):
            if (i == 7) or (i == 8) or (i == 6) or (i == 5):
                number_lower = (10 ** i) * -1
                number_higher = (10 ** (i + 1)) * -1
                interval_matches = less_valuable_matches[(less_valuable_matches['Value Difference (Abs)'] <= number_lower) & (less_valuable_matches['Value Difference (Abs)'] >= number_higher / 2)]
                num_matches = len(interval_matches)
                interval_wins = interval_matches[interval_matches['Team Won?'] == True]
                num_wins = len(interval_wins)
                win_percentage = (num_wins / num_matches) * 100 if num_matches > 0 else 0
                value_set = []
                for index, row in interval_matches.iterrows():
                    if row['Team Position'] == 'Team 1':
                        value_set.append(row['Team.1 Total Market Value'])
                    elif row['Team Position'] == 'Team 2':
                        value_set.append(row['Team.2 Total Market Value'])
                if value_set == []:
                    mean_mv_this_interval = 0
                    sd_mv_this_interval = 0
                    max_mv_this_interval = 0
                else:
                    mean_mv_this_interval = np.array(value_set).mean()
                    sd_mv_this_interval = np.array(value_set).std()
                    max_mv_this_interval = np.array(value_set).max()
                interval_row = {
                    'Team Name': first_team,
                    'Interval Range': f"${number_lower} - ${number_higher / 2}",
                    'Total Matches': num_matches,
                    'Total Wins': num_wins,
                    'Win Percentage': win_percentage,
                    'Mean MV, this interval': mean_mv_this_interval,
                    'SD MV, this interval': sd_mv_this_interval,
                    'Max MV, this interval': max_mv_this_interval
                }
                interval_stats_BELOW = pd.concat([interval_stats_BELOW, pd.DataFrame([interval_row])], ignore_index=True)

                interval_matches_ii = less_valuable_matches[(less_valuable_matches['Value Difference (Abs)'] <= (number_higher / 2)) & (less_valuable_matches['Value Difference (Abs)'] >= number_higher)]
                num_matches_ii = len(interval_matches_ii)
                interval_wins_ii = interval_matches_ii[interval_matches_ii['Team Won?'] == True]
                num_wins_ii = len(interval_wins_ii)
                win_percentage_ii = (num_wins_ii / num_matches_ii) * 100 if num_matches_ii > 0 else 0
                value_set_ii = []
                for index, row in interval_matches_ii.iterrows():
                    if row['Team Position'] == 'Team 1':
                        value_set_ii.append(row['Team.1 Total Market Value'])
                    elif row['Team Position'] == 'Team 2':
                        value_set_ii.append(row['Team.2 Total Market Value'])
                if value_set_ii == []:
                    mean_mv_this_interval = 0
                    sd_mv_this_interval = 0
                    max_mv_this_interval = 0
                else:
                    mean_mv_this_interval = np.array(value_set_ii).mean()
                    sd_mv_this_interval = np.array(value_set_ii).std()
                    max_mv_this_interval = np.array(value_set_ii).max()
                interval_row = {
                    'Team Name': first_team,
                    'Interval Range': f"${number_higher / 2} - ${number_higher}",
                    'Total Matches': num_matches_ii,
                    'Total Wins': num_wins_ii,
                    'Win Percentage': win_percentage_ii,
                    'Mean MV, this interval': mean_mv_this_interval,
                    'SD MV, this interval': sd_mv_this_interval,
                    'Max MV, this interval': max_mv_this_interval
                }
                interval_stats_BELOW = pd.concat([interval_stats_BELOW, pd.DataFrame([interval_row])], ignore_index=True)

            else:
                number_lower = (10 ** i) * -1
                number_higher = (10 ** (i + 1)) * -1
                interval_matches = less_valuable_matches[(less_valuable_matches['Value Difference (Abs)'] <= number_lower) & (less_valuable_matches['Value Difference (Abs)'] >= number_higher)]
                num_matches = len(interval_matches)
                interval_wins = interval_matches[interval_matches['Team Won?'] == True]
                num_wins = len(interval_wins)
                win_percentage = (num_wins / num_matches) * 100 if num_matches > 0 else 0
                value_set = []
                for index, row in interval_matches.iterrows():
                    if row['Team Position'] == 'Team 1':
                        value_set.append(row['Team.1 Total Market Value'])
                    elif row['Team Position'] == 'Team 2':
                        value_set.append(row['Team.2 Total Market Value'])
                if value_set == []:
                    mean_mv_this_interval = 0
                    sd_mv_this_interval = 0
                    max_mv_this_interval = 0
                else:
                    mean_mv_this_interval = np.array(value_set).mean()
                    sd_mv_this_interval = np.array(value_set).std()
                    max_mv_this_interval = np.array(value_set).max()
                interval_row = {
                    'Team Name': first_team,
                    'Interval Range': f"${number_lower} - ${number_higher / 2}",
                    'Total Matches': num_matches,
                    'Total Wins': num_wins,
                    'Win Percentage': win_percentage,
                    'Mean MV, this interval': mean_mv_this_interval,
                    'SD MV, this interval': sd_mv_this_interval,
                    'Max MV, this interval': max_mv_this_interval
                }
                interval_stats_BELOW = pd.concat([interval_stats_BELOW, pd.DataFrame([interval_row])], ignore_index=True)
        interval_stats_BELOW['Interval Side'] = 'Less Valuable'

    interval_stats_final = pd.concat([interval_stats_ABOVE, interval_stats_BELOW], ignore_index=True)
    mean_squad_value = (latest_match_data[latest_match_data['Team.2.Name'] == first_team]['Team.2.Total.Market.Value'].mean() + latest_match_data[latest_match_data['Team.1.Name'] == first_team]['Team.1.Total.Market.Value'].mean()) / 2

    mean_squad_value_difference = ((latest_match_data[latest_match_data['Team.2.Name'] == first_team]['Market.Value.Difference'].mean() * -1) + latest_match_data[latest_match_data['Team.1.Name'] == first_team]['Market.Value.Difference'].mean()) / 2

    avg_squad_value_difference_SD = ((latest_match_data[latest_match_data['Team.2.Name'] == first_team]['Market.Value.Difference'].std()) + latest_match_data[latest_match_data['Team.1.Name'] == first_team]['Market.Value.Difference'].std()) / 2

    max_mv = max(latest_match_data[latest_match_data['Team.2.Name'] == first_team]['Team.2.Total.Market.Value'].max(), latest_match_data[latest_match_data['Team.1.Name'] == first_team]['Team.1.Total.Market.Value'].max())

    min_mv = min(latest_match_data[latest_match_data['Team.2.Name'] == first_team]['Team.2.Total.Market.Value'].min(), latest_match_data[latest_match_data['Team.1.Name'] == first_team]['Team.1.Total.Market.Value'].min())

    max_mv_difference = max((latest_match_data[latest_match_data['Team.2.Name'] == first_team]['Market.Value.Difference'].min() * -1), latest_match_data[latest_match_data['Team.1.Name'] == first_team]['Market.Value.Difference'].max())

    max_mv_difference_lower = max((latest_match_data[latest_match_data['Team.2.Name'] == first_team]['Market.Value.Difference'].max()), latest_match_data[latest_match_data['Team.1.Name'] == first_team]['Market.Value.Difference'].min() * -1)

    new_row = {
        'Team Name': first_team,
        'Interval Range': "Total",
        'Total Matches': interval_stats_final['Total Matches'].sum(),
        'Total Wins': interval_stats_final['Total Wins'].sum(),
        'Win Percentage': interval_stats_final['Total Wins'].sum() / interval_stats_final['Total Matches'].sum(),
        'Mean MV, this interval': mean_squad_value,
        'SD MV, this interval': sd_mv_this_interval,
        'Max MV, this interval': max_mv_this_interval,
        'Interval Side': 'Both'
    }
    new_row_df = pd.DataFrame([new_row])
    interval_stats_final = pd.concat([interval_stats_final, new_row_df], ignore_index=True)
    #interval_stats_final['Avg Squad Value Difference vs Opponent'] = round(mean_squad_value_difference, 2)
    #interval_stats_final['SD, Squad Value Difference vs Opponent'] = round(avg_squad_value_difference_SD, 2)
    interval_stats_final['Max Squad MV'] = max_mv
    interval_stats_final['Min Squad MV'] = min_mv
    interval_stats_final['Max Squad MV Difference vs Opponent'] = max_mv_difference
    interval_stats_final['Min Squad MV Difference vs Opponent'] = max_mv_difference_lower * -1
 
    
    return interval_stats_final

### Creating Script (OLD)

In [ ]:
#####PAGE 1#####


#for each Unique Team Name in latest_match_data


#if the Team Name is Team.1.Name, add that Score.Difference and market Market.Value.Difference as a row to the table
    #add the opposing team name (Team.2.Name), add Team.1.Total.Market.Value, add Team.2.Total.Market.Value	
    #add the Date and Competition and Season

#if the Team Name is Team.2.Name, add (-1* that Score.Difference) and (-1 * market Market.Value.Difference) as a row to the table
    #add the opposing team name (Team.2.Name), add Team.1.Total.Market.Value, add Team.2.Total.Market.Value	
    #add the Date and Competition and Season

#you want a table with 
    #Country name (all rows will be the same)
    #Date 
    #Competition
    #Score.Difference
    #Market.Value.Difference
    #Team.1.Total.Market.Value, add Team.2.Total.Market.Value	
    #Season

#give me the code to do this for just latest_match_data['Team.1.Name'].unique()[0], the first team in the unique list of teams. I will turn it into a loop later. 

# Extract the first unique team name from 'Team.1.Name'
first_team = 'England' #latest_match_data['Team.1.Name'].unique()[0]

# Filter the rows where the team is either in 'Team.1.Name' or 'Team.2.Name'
team_data = latest_match_data[(latest_match_data['Team.1.Name'] == first_team) | (latest_match_data['Team.2.Name'] == first_team)]

# Create a new DataFrame to store the results
columns = ['Country Name', 'Date', 'Competition', 'Score Difference', 'Market Value Difference', 'Team.1 Total Market Value', 'Team.2 Total Market Value', 'Season', 'Team Position']
result_table = pd.DataFrame(columns=columns)

# Populate the DataFrame
for index, row in team_data.iterrows():
    if row['Team.1.Name'] == first_team:
        new_row = {
            'Country Name': first_team,
            'Date': row['Date'],
            'Competition': row['Competition'],
            'Score Difference': row['Score.Difference'],
            'Market Value Difference': row['Market.Value.Difference'],
            'Team.1 Total Market Value': row['Team.1.Total.Market.Value'],
            'Team.2 Total Market Value': row['Team.2.Total.Market.Value'],
            'Season': row['Season'],
            'Team Position': 'Team 1'
        }
    else:
        new_row = {
            'Country Name': first_team,
            'Date': row['Date'],
            'Competition': row['Competition'],
            'Score Difference': row['Score.Difference'],
            'Market Value Difference': row['Market.Value.Difference'],
            'Team.1 Total Market Value': row['Team.1.Total.Market.Value'],
            'Team.2 Total Market Value': row['Team.2.Total.Market.Value'],
            'Season': row['Season'],
            'Team Position': 'Team 2'
        }
    new_df = pd.DataFrame([new_row])
    new_df = new_df.dropna(axis=1, how='all')

    result_table = pd.concat([result_table, new_df], ignore_index=True)

result_table


In [ ]:
#####PAGE 2#####


# Add new columns with default values
result_table['Team Won?'] = False
result_table['Team More Valuable?'] = False

# Update the columns based on conditions
for index, row in result_table.iterrows():
    if row['Team Position'] == 'Team 1':
        if row['Score Difference'] > 0:
            result_table.at[index, 'Team Won?'] = True
        if row['Market Value Difference'] > 0:
            result_table.at[index, 'Team More Valuable?'] = True
    elif row['Team Position'] == 'Team 2':
        if row['Score Difference'] < 0:
            result_table.at[index, 'Team Won?'] = True
        if row['Market Value Difference'] < 0:
            result_table.at[index, 'Team More Valuable?'] = True

result_table

In [ ]:
#####PAGE 3#####

# Calculate the total number of matches
less_valuable_matches = result_table[result_table['Team More Valuable?'] == False]
more_valuable_matches = result_table[result_table['Team More Valuable?'] == True]
total_num_matches_more_valuable = len(more_valuable_matches)
if len(more_valuable_matches) > 0:
    more_valuable_matches['Value Difference (Abs)'] = abs(result_table['Market Value Difference'])
else:
    0==0
total_num_matches_less_valuable = len(less_valuable_matches)
if len(less_valuable_matches) > 0:
    less_valuable_matches['Value Difference (Abs)'] = abs(result_table['Market Value Difference']) * -1

In [ ]:
#####PAGE 4#####

#PRODUCING TABLE FROM MORE VALUABLE MATCHES

# Create a DataFrame to store the results
interval_stats = pd.DataFrame(columns=['Team Name', 'Interval Range', 'Total Matches', 'Total Wins', 'Win Percentage'])
for i in range(5, 9):
    if (i == 7) or (i == 8) or (i == 6):
        number_lower = 10 ** i
        number_higher = 10 ** (i + 1)
        interval_matches = more_valuable_matches[(more_valuable_matches['Value Difference (Abs)'] >= number_lower) & (more_valuable_matches['Value Difference (Abs)'] <= number_higher / 2)]
        num_matches = len(interval_matches)
        interval_wins = interval_matches[interval_matches['Team Won?'] == True]
        num_wins = len(interval_wins)
        win_percentage = (num_wins / num_matches) * 100 if num_matches > 0 else 0
        interval_row = {
            'Team Name': first_team,
            'Interval Range': f"${number_lower} - ${number_higher / 2}",
            'Total Matches': num_matches,
            'Total Wins': num_wins,
            'Win Percentage': win_percentage
        }
        interval_stats = pd.concat([interval_stats, pd.DataFrame([interval_row])], ignore_index=True)

        interval_matches_ii = more_valuable_matches[(more_valuable_matches['Value Difference (Abs)'] >= (number_higher / 2)) & (more_valuable_matches['Value Difference (Abs)'] <= number_higher)]
        num_matches_ii = len(interval_matches_ii)
        interval_wins_ii = interval_matches_ii[interval_matches_ii['Team Won?'] == True]
        num_wins_ii = len(interval_wins_ii)
        win_percentage_ii = (num_wins_ii / num_matches_ii) * 100 if num_matches_ii > 0 else 0
        interval_row = {
            'Team Name': first_team,
            'Interval Range': f"${number_higher / 2} - ${number_higher}",
            'Total Matches': num_matches_ii,
            'Total Wins': num_wins_ii,
            'Win Percentage': win_percentage_ii
        }
        interval_stats = pd.concat([interval_stats, pd.DataFrame([interval_row])], ignore_index=True)

    else:
        number_lower = 10 ** i
        number_higher = 10 ** (i + 1)
        interval_matches = more_valuable_matches[(more_valuable_matches['Value Difference (Abs)'] >= number_lower) & (more_valuable_matches['Value Difference (Abs)'] <= number_higher)]
        num_matches = len(interval_matches)
        interval_wins = interval_matches[interval_matches['Team Won?'] == True]
        num_wins = len(interval_wins)
        win_percentage = (num_wins / num_matches) * 100 if num_matches > 0 else 0
        interval_row = {
            'Team Name': first_team,
            'Interval Range': f"${number_lower} - ${number_higher / 2}",
            'Total Matches': num_matches,
            'Total Wins': num_wins,
            'Win Percentage': win_percentage
        }
        interval_stats = pd.concat([interval_stats, pd.DataFrame([interval_row])], ignore_index=True)
interval_stats['Interval Side'] = 'More Valuable'
interval_stats

In [ ]:
#####PAGE 5#####

#PRODUCING TABLE FROM LESS VALUABLE MATCHES

if len(less_valuable_matches) == 0:
    interval_stats = pd.DataFrame(columns=['Team Name', 'Interval Range', 'Total Matches', 'Total Wins', 'Win Percentage'])
else:
    0==0

    # Create a DataFrame to store the results
    interval_stats = pd.DataFrame(columns=['Team Name', 'Interval Range', 'Total Matches', 'Total Wins', 'Win Percentage'])
    for i in range(5, 9):
        if (i == 7) or (i == 8) or (i == 6):
            number_lower = (10 ** i) * -1
            number_higher = (10 ** (i + 1)) * -1
            interval_matches = less_valuable_matches[(less_valuable_matches['Value Difference (Abs)'] <= number_lower) & (less_valuable_matches['Value Difference (Abs)'] >= number_higher / 2)]
            num_matches = len(interval_matches)
            interval_wins = interval_matches[interval_matches['Team Won?'] == True]
            num_wins = len(interval_wins)
            win_percentage = (num_wins / num_matches) * 100 if num_matches > 0 else 0
            interval_row = {
                'Team Name': first_team,
                'Interval Range': f"${number_lower} - ${number_higher / 2}",
                'Total Matches': num_matches,
                'Total Wins': num_wins,
                'Win Percentage': win_percentage
            }
            interval_stats = pd.concat([interval_stats, pd.DataFrame([interval_row])], ignore_index=True)

            interval_matches_ii = less_valuable_matches[(less_valuable_matches['Value Difference (Abs)'] <= (number_higher / 2)) & (less_valuable_matches['Value Difference (Abs)'] >= number_higher)]
            num_matches_ii = len(interval_matches_ii)
            interval_wins_ii = interval_matches_ii[interval_matches_ii['Team Won?'] == True]
            num_wins_ii = len(interval_wins_ii)
            win_percentage_ii = (num_wins_ii / num_matches_ii) * 100 if num_matches_ii > 0 else 0
            interval_row = {
                'Team Name': first_team,
                'Interval Range': f"${number_higher / 2} - ${number_higher}",
                'Total Matches': num_matches_ii,
                'Total Wins': num_wins_ii,
                'Win Percentage': win_percentage_ii
            }
            interval_stats = pd.concat([interval_stats, pd.DataFrame([interval_row])], ignore_index=True)

        else:
            number_lower = (10 ** i) * -1
            number_higher = (10 ** (i + 1)) * -1
            interval_matches = less_valuable_matches[(less_valuable_matches['Value Difference (Abs)'] <= number_lower) & (less_valuable_matches['Value Difference (Abs)'] >= number_higher)]
            num_matches = len(interval_matches)
            interval_wins = interval_matches[interval_matches['Team Won?'] == True]
            num_wins = len(interval_wins)
            win_percentage = (num_wins / num_matches) * 100 if num_matches > 0 else 0
            interval_row = {
                'Team Name': first_team,
                'Interval Range': f"${number_lower} - ${number_higher / 2}",
                'Total Matches': num_matches,
                'Total Wins': num_wins,
                'Win Percentage': win_percentage
            }
            interval_stats = pd.concat([interval_stats, pd.DataFrame([interval_row])], ignore_index=True)
    interval_stats['Interval Side'] = 'Less Valuable'
interval_stats

In [ ]:
#####NOT A PAGE#####

# Calculate the total number of matches
less_valuable_matches = result_table[result_table['Team More Valuable?'] == False]
more_valuable_matches = result_table[result_table['Team More Valuable?'] == True]
total_num_matches_more_valuable = len(more_valuable_matches)
more_valuable_matches['Value Difference (Abs)'] = abs(result_table['Market Value Difference'])

for i in range(5, 9):
    if (i == 7) or (i == 8) or (i == 6):
        print(i, '**')
        number_lower = 10 ** i
        number_higher = 10 ** (i + 1)
        interval_matches = more_valuable_matches[(more_valuable_matches['Value Difference (Abs)'] >= number_lower) & (more_valuable_matches['Value Difference (Abs)'] <= number_higher / 2)]
        num_matches = len(interval_matches)
        interval_wins = interval_matches[interval_matches['Team Won?'] == True]
        num_wins = len(interval_wins)
        print(number_lower, number_higher / 2, num_matches, num_wins)
        interval_matches_ii = more_valuable_matches[(more_valuable_matches['Value Difference (Abs)'] >= (number_higher / 2)) & (more_valuable_matches['Value Difference (Abs)'] <= number_higher)]
        num_matches_ii = len(interval_matches_ii)
        interval_wins_ii = interval_matches_ii[interval_matches_ii['Team Won?'] == True]
        num_wins_ii = len(interval_wins_ii)
        if num_matches_ii != 0:
            print((number_higher / 2), number_higher, num_matches_ii, num_wins_ii, num_wins_ii / num_matches_ii)
        else:
            print((number_higher / 2), number_higher, num_matches_ii, num_wins_ii, 0)
    else:
        print(i)
        number_lower = 10 ** i
        number_higher = 10 ** (i + 1)
        interval_matches = more_valuable_matches[(more_valuable_matches['Value Difference (Abs)'] >= number_lower) & (more_valuable_matches['Value Difference (Abs)'] <= number_higher)]
        num_matches = len(interval_matches)
        interval_wins = interval_matches[interval_matches['Team Won?'] == True]
        num_wins = len(interval_wins)
        print(number_lower, number_higher, num_matches, num_wins, num_wins / num_matches)
        if num_matches != 0:
            print((number_higher / 2), number_higher, num_matches, num_wins, num_wins/ num_matches)
        else:
            print((number_higher / 2), number_higher, num_matches_ii, num_wins_ii, 0)


## Task 2 - UEFA vs everyone else

In [8]:
# Filter the DataFrame to find matches where one team is from UEFA and the other is not
uefa_vs_others = latest_match_data[
    ((latest_match_data['Team.1.Confederation'] == 'UEFA') & (latest_match_data['Team.2.Confederation'] != 'UEFA')) |
    ((latest_match_data['Team.2.Confederation'] == 'UEFA') & (latest_match_data['Team.1.Confederation'] != 'UEFA'))
]


uefa_vs_uefa = latest_match_data[
    ((latest_match_data['Team.1.Confederation'] == 'UEFA') & (latest_match_data['Team.2.Confederation'] == 'UEFA'))]

In [9]:
uefa_vs_others['Competition'].value_counts()


Competition
World Cup    117
Olympics      30
Name: count, dtype: int64

In [10]:
uefa_vs_uefa['Competition'].value_counts()

Competition
World Cup Qualifiers, Europe    780
Euros, Qualifiers               740
Nations League                  454
Euros                           102
World Cup                        29
Olympics                          1
Name: count, dtype: int64

In [11]:
#for each team in all_teams_uefa
 
#their win PCT when they play less valuable teams that are in UEFA 
#their win PCT when they play more valuable teams that are in UEFA

#their win PCT when they play less valuable teams that are NOT in UEFA 
#their win PCT when they play more valuable teams that are NOT in UEFA

set_team_1 = uefa_vs_others[uefa_vs_others['Team.1.Confederation'] == 'UEFA']['Team.1.Name'].unique()
set_team_2 = uefa_vs_others[uefa_vs_others['Team.2.Confederation'] == 'UEFA']['Team.2.Name'].unique()

all_teams_uefa = np.union1d(set_team_1, set_team_2)

In [ ]:
uefa_more_team_1_rows = team_matches_vs_uefa_more_valuable[team_matches_vs_uefa_more_valuable['Team.1.Name'] == team]
uefa_more_team_2_rows = team_matches_vs_uefa_more_valuable[team_matches_vs_uefa_more_valuable['Team.2.Name'] == team]
opponent_mkt_values = []
total_games = len(team_matches_vs_uefa_more_valuable)
for index, row in uefa_more_team_2_rows.iterrows():
    opponent_mkt_values.append(row['Team.1.Total.Market.Value'])
for index, row in uefa_more_team_1_rows.iterrows():
    opponent_mkt_values.append(row['Team.2.Total.Market.Value'])
mean_mv_opponent = np.array(opponent_mkt_values).mean()
sd_mv_opponent = np.array(opponent_mkt_values).std()
max_mv_opponent = np.array(opponent_mkt_values).max()
min_mv_opponent = np.array(opponent_mkt_values).min()

mean_mv_opponent, sd_mv_opponent, max_mv_opponent, min_mv_opponent

In [17]:
win_pct_stats = pd.DataFrame()
for team in all_teams_uefa:
    # Filter matches involving the team
    team_matches_vs_not_uefa = uefa_vs_others[(uefa_vs_others['Team.1.Name'] == team) | (uefa_vs_others['Team.2.Name'] == team)]
    team_matches_vs_uefa = uefa_vs_uefa[(uefa_vs_uefa['Team.1.Name'] == team) | (uefa_vs_uefa['Team.2.Name'] == team)]


    num_matches_vs_not_uefa = len(team_matches_vs_not_uefa)
    num_matches_vs_uefa = len(team_matches_vs_uefa)


    #PLAYING NON-UEFA CONFEDERATIONS
    team_matches_vs_not_uefa_more_valuable = team_matches_vs_not_uefa[
        ((team_matches_vs_not_uefa['Team.1.Name'] == team) & (team_matches_vs_not_uefa['Team.1.Total.Market.Value'] > team_matches_vs_not_uefa['Team.2.Total.Market.Value'])) |
        ((team_matches_vs_not_uefa['Team.2.Name'] == team) & (team_matches_vs_not_uefa['Team.1.Total.Market.Value'] < team_matches_vs_not_uefa['Team.2.Total.Market.Value']))
    ]
    team_matches_vs_not_uefa_less_valuable = team_matches_vs_not_uefa[
        ((team_matches_vs_not_uefa['Team.1.Name'] == team) & (team_matches_vs_not_uefa['Team.1.Total.Market.Value'] < team_matches_vs_not_uefa['Team.2.Total.Market.Value'])) |
        ((team_matches_vs_not_uefa['Team.2.Name'] == team) & (team_matches_vs_not_uefa['Team.1.Total.Market.Value'] > team_matches_vs_not_uefa['Team.2.Total.Market.Value']))
    ]

    wins_vs_not_uefa_more_valuable = team_matches_vs_not_uefa_more_valuable[
        ((team_matches_vs_not_uefa_more_valuable['Team.1.Name'] == team) & (team_matches_vs_not_uefa_more_valuable['Score.Difference'] > 0)) |
        ((team_matches_vs_not_uefa_more_valuable['Team.2.Name'] == team) & (team_matches_vs_not_uefa_more_valuable['Score.Difference'] < 0))
    ]

    wins_vs_not_uefa_less_valuable = team_matches_vs_not_uefa_less_valuable[
        ((team_matches_vs_not_uefa_less_valuable['Team.1.Name'] == team) & (team_matches_vs_not_uefa_less_valuable['Score.Difference'] > 0)) |
        ((team_matches_vs_not_uefa_less_valuable['Team.2.Name'] == team) & (team_matches_vs_not_uefa_less_valuable['Score.Difference'] < 0))
    ]

    number_matches_vs_not_uefa_more_valuable = len(team_matches_vs_not_uefa_more_valuable)
    number_matches_vs_not_uefa_less_valuable = len(team_matches_vs_not_uefa_less_valuable)
    number_of_wins_vs_not_uefa_more_valuable = len(wins_vs_not_uefa_more_valuable)
    number_of_wins_vs_not_uefa_less_valuable = len(wins_vs_not_uefa_less_valuable)

    #PLAYING UEFA
    team_matches_vs_uefa_more_valuable = team_matches_vs_uefa[
        ((team_matches_vs_uefa['Team.1.Name'] == team) & (team_matches_vs_uefa['Team.1.Total.Market.Value'] > team_matches_vs_uefa['Team.2.Total.Market.Value'])) |
        ((team_matches_vs_uefa['Team.2.Name'] == team) & (team_matches_vs_uefa['Team.1.Total.Market.Value'] < team_matches_vs_uefa['Team.2.Total.Market.Value']))
    ]
    team_matches_vs_uefa_less_valuable = team_matches_vs_uefa[
        ((team_matches_vs_uefa['Team.1.Name'] == team) & (team_matches_vs_uefa['Team.1.Total.Market.Value'] < team_matches_vs_uefa['Team.2.Total.Market.Value'])) |
        ((team_matches_vs_uefa['Team.2.Name'] == team) & (team_matches_vs_uefa['Team.1.Total.Market.Value'] > team_matches_vs_uefa['Team.2.Total.Market.Value']))
    ]

    wins_vs_uefa_more_valuable = team_matches_vs_uefa_more_valuable[
        ((team_matches_vs_uefa_more_valuable['Team.1.Name'] == team) & (team_matches_vs_uefa_more_valuable['Score.Difference'] > 0)) |
        ((team_matches_vs_uefa_more_valuable['Team.2.Name'] == team) & (team_matches_vs_uefa_more_valuable['Score.Difference'] < 0))
    ]

    wins_vs_uefa_less_valuable = team_matches_vs_uefa_less_valuable[
        ((team_matches_vs_uefa_less_valuable['Team.1.Name'] == team) & (team_matches_vs_uefa_less_valuable['Score.Difference'] > 0)) |
        ((team_matches_vs_uefa_less_valuable['Team.2.Name'] == team) & (team_matches_vs_uefa_less_valuable['Score.Difference'] < 0))
    ]

    number_of_matches_vs_uefa_more_valuable = len(team_matches_vs_uefa_more_valuable)
    number_of_matches_vs_uefa_less_valuable = len(team_matches_vs_uefa_less_valuable)
    number_of_wins_vs_uefa_more_valuable = len(wins_vs_uefa_more_valuable)
    number_of_wins_vs_uefa_less_valuable = len(wins_vs_uefa_less_valuable)

    #PERCENTAGES
    #VS UEFA - MORE VALUABLE
    if number_of_matches_vs_uefa_more_valuable != 0:
        win_pct_more_val_uefa = number_of_wins_vs_uefa_more_valuable / number_of_matches_vs_uefa_more_valuable
    
        uefa_more_team_1_rows = team_matches_vs_uefa_more_valuable[team_matches_vs_uefa_more_valuable['Team.1.Name'] == team]
        uefa_more_team_2_rows = team_matches_vs_uefa_more_valuable[team_matches_vs_uefa_more_valuable['Team.2.Name'] == team]
        team_mkt_values = []
        opponent_mkt_values = []
        game_differences = []
        total_games = len(team_matches_vs_uefa_more_valuable)
        for index, row in uefa_more_team_2_rows.iterrows():
            opponent_mkt_values.append(row['Team.1.Total.Market.Value'])
            team_mkt_values.append(row['Team.2.Total.Market.Value'])
            game_differences.append(abs(row['Market.Value.Difference']))
        for index, row in uefa_more_team_1_rows.iterrows():
            opponent_mkt_values.append(row['Team.2.Total.Market.Value'])
            team_mkt_values.append(row['Team.1.Total.Market.Value'])
            game_differences.append(row['Market.Value.Difference'])
        mean_mv_opponent_more_uefa = np.array(opponent_mkt_values).mean()
        sd_mv_opponent_more_uefa = np.array(opponent_mkt_values).std()
        max_mv_opponent_more_uefa = np.array(opponent_mkt_values).max()
        min_mv_opponent_more_uefa = np.array(opponent_mkt_values).min()
        mean_squad_mv_more_uefa = np.array(team_mkt_values).mean()
        mean_mv_difference_more_uefa = np.array(game_differences).mean()
    else:
        win_pct_more_val_uefa = 0
        mean_mv_opponent_more_uefa = 0
        sd_mv_opponent_more_uefa = 0
        max_mv_opponent_more_uefa = 0
        min_mv_opponent_more_uefa = 0
        mean_squad_mv_more_uefa = 0
        mean_mv_difference_more_uefa = 0
    
    #VS UEFA - LESS VALUABLE
    if number_of_matches_vs_uefa_less_valuable != 0:
        win_pct_less_val_uefa = number_of_wins_vs_uefa_less_valuable / number_of_matches_vs_uefa_less_valuable

        uefa_more_team_1_rows = team_matches_vs_uefa_less_valuable[team_matches_vs_uefa_less_valuable['Team.1.Name'] == team]
        uefa_more_team_2_rows = team_matches_vs_uefa_less_valuable[team_matches_vs_uefa_less_valuable['Team.2.Name'] == team]
        team_mkt_values = []
        opponent_mkt_values = []
        game_differences = []
        total_games = len(team_matches_vs_uefa_less_valuable)
        for index, row in uefa_more_team_2_rows.iterrows():
            opponent_mkt_values.append(row['Team.1.Total.Market.Value'])
            team_mkt_values.append(row['Team.2.Total.Market.Value'])
            game_differences.append((row['Market.Value.Difference']))
        for index, row in uefa_more_team_1_rows.iterrows():
            opponent_mkt_values.append(row['Team.2.Total.Market.Value'])
            team_mkt_values.append(row['Team.1.Total.Market.Value'])
            game_differences.append(abs(row['Market.Value.Difference']))
        mean_mv_opponent_less_uefa = np.array(opponent_mkt_values).mean()
        sd_mv_opponent_less_uefa = np.array(opponent_mkt_values).std()
        max_mv_opponent_less_uefa = np.array(opponent_mkt_values).max()
        min_mv_opponent_less_uefa = np.array(opponent_mkt_values).min()
        mean_squad_mv_less_uefa = np.array(team_mkt_values).mean()
        mean_mv_difference_less_uefa = np.array(game_differences).mean()
    else:
        win_pct_less_val_uefa = 0
        mean_mv_opponent_less_uefa = 0
        sd_mv_opponent_less_uefa = 0
        max_mv_opponent_less_uefa = 0
        min_mv_opponent_less_uefa = 0
        mean_squad_mv_less_uefa = 0
        mean_mv_difference_less_uefa = 0
    
    #VS NOT UEFA - less valuable
    if number_matches_vs_not_uefa_less_valuable != 0:
        win_pct_less_val_non_uefa = number_of_wins_vs_not_uefa_less_valuable / number_matches_vs_not_uefa_less_valuable
        uefa_more_team_1_rows = team_matches_vs_not_uefa_less_valuable[team_matches_vs_not_uefa_less_valuable['Team.1.Name'] == team]
        uefa_more_team_2_rows = team_matches_vs_not_uefa_less_valuable[team_matches_vs_not_uefa_less_valuable['Team.2.Name'] == team]
        team_mkt_values = []
        opponent_mkt_values = []
        game_differences = []
        total_games = len(team_matches_vs_not_uefa_less_valuable)
        for index, row in uefa_more_team_2_rows.iterrows():
            opponent_mkt_values.append(row['Team.1.Total.Market.Value'])
            team_mkt_values.append(row['Team.2.Total.Market.Value'])
            game_differences.append((row['Market.Value.Difference']))
        for index, row in uefa_more_team_1_rows.iterrows():
            opponent_mkt_values.append(row['Team.2.Total.Market.Value'])
            team_mkt_values.append(row['Team.1.Total.Market.Value'])
            game_differences.append(abs(row['Market.Value.Difference']))
        mean_mv_opponent_less_non_uefa = np.array(opponent_mkt_values).mean()
        sd_mv_opponent_less_non_uefa = np.array(opponent_mkt_values).std()
        max_mv_opponent_less_non_uefa = np.array(opponent_mkt_values).max()
        min_mv_opponent_less_non_uefa = np.array(opponent_mkt_values).min()
        mean_squad_mv_less_non_uefa = np.array(team_mkt_values).mean()
        mean_mv_difference_not_uefa_less = np.array(game_differences).mean()
    else:
        win_pct_less_val_less_non_uefa = 0
        mean_mv_opponent_less_non_uefa = 0
        sd_mv_opponent_less_non_uefa = 0
        max_mv_opponent_less_non_uefa = 0
        min_mv_opponent_less_non_uefa = 0
        mean_squad_mv_less_non_uefa = 0
        mean_mv_difference_not_uefa_less = 0
    
    #VS NOT UEFA - more valuable 
    if number_matches_vs_not_uefa_more_valuable != 0:
        win_pct_more_val_non_uefa = number_of_wins_vs_not_uefa_more_valuable / number_matches_vs_not_uefa_more_valuable
        uefa_more_team_1_rows = team_matches_vs_not_uefa_more_valuable[team_matches_vs_not_uefa_more_valuable['Team.1.Name'] == team]
        uefa_more_team_2_rows = team_matches_vs_not_uefa_more_valuable[team_matches_vs_not_uefa_more_valuable['Team.2.Name'] == team]
        team_mkt_values = []
        opponent_mkt_values = []
        game_differences = []
        total_games = len(team_matches_vs_not_uefa_more_valuable)
        for index, row in uefa_more_team_2_rows.iterrows():
            opponent_mkt_values.append(row['Team.1.Total.Market.Value'])
            team_mkt_values.append(row['Team.2.Total.Market.Value'])
            game_differences.append(abs(row['Market.Value.Difference']))
        for index, row in uefa_more_team_1_rows.iterrows():
            opponent_mkt_values.append(row['Team.2.Total.Market.Value'])
            team_mkt_values.append(row['Team.1.Total.Market.Value'])
            game_differences.append(row['Market.Value.Difference'])
        mean_mv_opponent_more_non_uefa = np.array(opponent_mkt_values).mean()
        sd_mv_opponent_more_non_uefa = np.array(opponent_mkt_values).std()
        max_mv_opponent_more_non_uefa = np.array(opponent_mkt_values).max()
        min_mv_opponent_more_non_uefa = np.array(opponent_mkt_values).min()
        mean_squad_mv_more_non_uefa = np.array(team_mkt_values).mean()
        mean_mv_difference_not_uefa_more = np.array(game_differences).mean()
    else:
        win_pct_more_val_non_uefa = 0
        mean_mv_opponent_more_non_uefa = 0
        sd_mv_opponent_more_non_uefa = 0
        max_mv_opponent_more_non_uefa = 0
        min_mv_opponent_more_non_uefa = 0
        mean_squad_mv_more_non_uefa = 0
        mean_mv_difference_not_uefa_more = 0
    
    win_stats = {
        'Team Name': team,
        'Number of Matches vs UEFA as the More Valuable team': number_of_matches_vs_uefa_more_valuable,
        'Number of Wins vs UEFA as the More Valuable team': number_of_wins_vs_uefa_more_valuable,
        'Win PCT vs UEFA as the More Valuable team': win_pct_more_val_uefa,
        'Vs UEFA as more valuable team - Avg opponent squad MV': mean_mv_opponent_more_uefa,
        'Vs UEFA as more valuable team - SD, opponent squad MV': sd_mv_opponent_more_uefa,
        'Vs UEFA as more valuable team - max opponent squad MV': max_mv_opponent_more_uefa, 
        'Vs UEFA as more valuable team - min opponent squad MV': min_mv_opponent_more_uefa,
        'Vs UEFA as more valuable team - Avg squad MV': mean_squad_mv_more_uefa,
        'Vs UEFA as more valuable team - Avg MV difference': mean_mv_difference_more_uefa,

        'Number of Matches vs UEFA as the Less Valuable team': number_of_matches_vs_uefa_less_valuable,
        'Number of Wins vs UEFA as the Less Valuable team': number_of_wins_vs_uefa_less_valuable,
        'Win PCT vs UEFA as the Less Valuable team': win_pct_less_val_uefa,
        'Vs UEFA as less valuable team - Avg opponent squad MV': mean_mv_opponent_less_uefa,
        'Vs UEFA as less valuable team - SD, opponent squad MV': sd_mv_opponent_less_uefa,
        'Vs UEFA as less valuable team - max opponent squad MV': max_mv_opponent_less_uefa, 
        'Vs UEFA as less valuable team - min opponent squad MV': min_mv_opponent_less_uefa,
        'Vs UEFA as less valuable team - Avg squad MV': mean_squad_mv_less_uefa,
        'Vs UEFA as less valuable team - Avg MV difference': mean_mv_difference_less_uefa,

        'Number of Matches vs Non-UEFA as the More Valuable team': number_matches_vs_not_uefa_more_valuable,
        'Number of Wins vs Non-UEFA as the More Valuable team': number_of_wins_vs_not_uefa_more_valuable,
        'Win PCT vs Non-UEFA as the More Valuable team': win_pct_more_val_non_uefa,
        'Vs non-UEFA as more valuable team - Avg opponent squad MV': mean_mv_opponent_more_non_uefa,
        'Vs non-UEFA as more valuable team - SD, opponent squad MV': sd_mv_opponent_more_non_uefa,
        'Vs non-UEFA as more valuable team - max opponent squad MV': max_mv_opponent_more_non_uefa, 
        'Vs non-UEFA as more valuable team - min opponent squad MV': min_mv_opponent_more_non_uefa,
        'Vs non-UEFA as more valuable team - Avg squad MV': mean_squad_mv_more_non_uefa,
        'Vs non-UEFA as more valuable team - Avg MV difference': mean_mv_difference_not_uefa_more,

        
        'Number of Matches vs Non-UEFA as the Less Valuable team': number_matches_vs_not_uefa_less_valuable,
        'Number of Wins vs Non-UEFA as the Less Valuable team': number_of_wins_vs_not_uefa_less_valuable,
        'Win PCT vs Non-UEFA as the Less Valuable team': win_pct_less_val_non_uefa,
        'Vs non-UEFA as less valuable team - Avg opponent squad MV': mean_mv_opponent_less_non_uefa,
        'Vs non-UEFA as less valuable team - SD, opponent squad MV': sd_mv_opponent_less_non_uefa,
        'Vs non-UEFA as less valuable team - max opponent squad MV': max_mv_opponent_less_non_uefa, 
        'Vs non-UEFA as less valuable team - min opponent squad MV': min_mv_opponent_less_non_uefa,
        'Vs non-UEFA as less valuable team - Avg squad MV': mean_squad_mv_less_non_uefa,
        'Vs non-UEFA as less valuable team - Avg MV difference': mean_mv_difference_not_uefa_less
    }
    win_stats_df = pd.DataFrame([win_stats])
    # Append to DataFrame
    win_pct_stats = pd.concat([win_pct_stats, win_stats_df], ignore_index=True)
win_pct_stats

,Team Name,Number of Matches vs UEFA as the More Valuable team,Number of Wins vs UEFA as the More Valuable team,Win PCT vs UEFA as the More Valuable team,Vs UEFA as more valuable team - Avg opponent squad MV,"Vs UEFA as more valuable team - SD, opponent squad MV",Vs UEFA as more valuable team - max opponent squad MV,Vs UEFA as more valuable team - min opponent squad MV,Vs UEFA as more valuable team - Avg squad MV,Vs UEFA as more valuable team - Avg MV difference,Number of Matches vs UEFA as the Less Valuable team,Number of Wins vs UEFA as the Less Valuable team,Win PCT vs UEFA as the Less Valuable team,Vs UEFA as less valuable team - Avg opponent squad MV,"Vs UEFA as less valuable team - SD, opponent squad MV",Vs UEFA as less valuable team - max opponent squad MV,Vs UEFA as less valuable team - min opponent squad MV,Vs UEFA as less valuable team - Avg squad MV,Vs UEFA as less valuable team - Avg MV difference,Number of Matches vs Non-UEFA as the More Valuable team,Number of Wins vs Non-UEFA as the More Valuable team,Win PCT vs Non-UEFA as the More Valuable team,Vs non-UEFA as more valuable team - Avg opponent squad MV,"Vs non-UEFA as more valuable team - SD, opponent squad MV",Vs non-UEFA as more valuable team - max opponent squad MV,Vs non-UEFA as more valuable team - min opponent squad MV,Vs non-UEFA as more valuable team - Avg squad MV,Vs non-UEFA as more valuable team - Avg MV difference,Number of Matches vs Non-UEFA as the Less Valuable team,Number of Wins vs Non-UEFA as the Less Valuable team,Win PCT vs Non-UEFA as the Less Valuable team,Vs non-UEFA as less valuable team - Avg opponent squad MV,"Vs non-UEFA as less valuable team - SD, opponent squad MV",Vs non-UEFA as less valuable team - max opponent squad MV,Vs non-UEFA as less valuable team - min opponent squad MV,Vs non-UEFA as less valuable team - Avg squad MV,Vs non-UEFA as less valuable team - Avg MV difference
0,Belgium,79,63,0.797468,7.682767e+07,7.280525e+07,346500000.0,150000.0,3.095603e+08,2.327326e+08,8,4,0.500000,4.694250e+08,1.798546e+08,807000000.0,173500000.0,3.637025e+08,1.057225e+08,9,8,0.888889,1.035633e+08,1.242774e+08,408000000.0,4070000.0,3.846444e+08,2.810811e+08,1,0,0.000000,2.895000e+08,0.000000e+00,289500000.0,289500000.0,2.576000e+08,3.190000e+07
1,Bosnia,50,30,0.600000,2.265350e+07,2.313666e+07,91800000.0,175000.0,8.130340e+07,5.864990e+07,24,2,0.083333,2.868438e+08,1.602042e+08,639000000.0,82050000.0,7.139042e+07,2.154533e+08,2,1,0.500000,3.837500e+07,2.482500e+07,63200000.0,13550000.0,9.665000e+07,5.827500e+07,1,0,0.000000,2.705000e+08,0.000000e+00,270500000.0,270500000.0,1.064500e+08,1.640500e+08
2,Croatia,62,40,0.645161,6.745548e+07,5.397524e+07,232000000.0,1900000.0,1.774065e+08,1.099510e+08,24,7,0.291667,4.140938e+08,1.975946e+08,807000000.0,125500000.0,1.912458e+08,2.228479e+08,6,5,0.833333,1.119408e+08,5.318302e+07,198050000.0,47850000.0,1.995333e+08,8.759250e+07,5,2,0.400000,3.664400e+08,1.303650e+08,590000000.0,219000000.0,2.165800e+08,1.498600e+08
3,Denmark,52,35,0.673077,4.703404e+07,5.135702e+07,182000000.0,215000.0,1.464502e+08,9.941615e+07,29,10,0.344828,2.832695e+08,1.924576e+08,617000000.0,38650000.0,1.324890e+08,1.507805e+08,6,2,0.333333,2.098167e+07,1.199282e+07,34650000.0,4500000.0,1.535217e+08,1.325400e+08,2,0,0.000000,1.018000e+08,8.620000e+07,188000000.0,15600000.0,6.865000e+06,9.493500e+07
4,England,90,58,0.644444,1.332728e+08,1.520258e+08,612000000.0,25000.0,4.552822e+08,3.220095e+08,0,0,0.000000,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,8,5,0.625000,8.600000e+07,6.904375e+07,172900000.0,4070000.0,5.567125e+08,4.707125e+08,0,0,0.000000,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00
5,France,74,50,0.675676,1.368373e+08,1.501410e+08,563000000.0,735000.0,5.017209e+08,3.648837e+08,5,2,0.400000,4.210800e+08,1.289314e+08,677000000.0,336000000.0,3.124500e+08,1.086300e+08,14,9,0.642857,1.109500e+08,1.231344e+08,422000000.0,8500000.0,4.620036e+08,3.510536e+08,0,0,0.000000,0.000000e+00,0.000000e+0

## Task 3: Identifying Star Players

In [ ]:
market_value_data = load_csv_dataset('create_dataset_1/CSVs we use often/MarketValuesCompleteMay17.csv')
def convert_market_value(value):
    if value == '-':
        return 0
    if pd.isna(value):
        print(value)
        return 0
    if type(value) == float:
        return value
    else:
        value = value.replace('€', '')
        if 'm' in value:
            return float(value.replace('m', '')) * 1_000_000
        elif 'k' in value:
            return float(value.replace('k', '')) * 1_000
        return float(value)
# Apply the conversion function to the 'Market Value' column
market_value_data['Market Value Numeric'] = market_value_data['Market Value'].apply(convert_market_value)

In [ ]:
def plot_data_mv(market_value_data):
    # Function to format the tick labels as integers
    def format_func(value, tick_number):
        return f'{int(value)}'

# Assuming market_value_data is your DataFrame and 'Market Value' is the column
    #market_value_data['Market Value Numeric'].hist(bins=20)  # You can adjust the number of bins as needed

# Calculate the 80th and 90th percentiles
    percentile_95 = market_value_data['Market Value Numeric'].quantile(0.95)
    percentile_99 = market_value_data['Market Value Numeric'].quantile(0.99)

# Setting up the X-axis and Y-axis ticks to display full numbers
    # ax = plt.gca()
    # ax.xaxis.set_major_formatter(ticker.FuncFormatter(format_func))
    # ax.yaxis.set_major_formatter(ticker.FuncFormatter(format_func))
    # ax.xaxis.set_major_locator(ticker.MaxNLocator(5))  # Adjust the number of ticks as needed
    # ax.set_yscale('log')  # Set the Y-axis to a logarithmic scale

# Add vertical lines for the 80th and 90th percentiles
    #ax.axvline(percentile_95, color='red', linestyle='dashed', linewidth=1, label=f'95th percentile ({int(percentile_95)})')
    #ax.axvline(percentile_99, color='blue', linestyle='dashed', linewidth=1, label=f'99th percentile ({int(percentile_99)})')

    #plt.title('Histogram of Market Value')
    #plt.xlabel('Market Value')
    #plt.ylabel('Frequency')
    #plt.legend()
    #plt.show()
    return percentile_95, percentile_99

In [ ]:
def plot_data_mv_confederation(market_value_data):
    # Function to format the tick labels as integers
    def format_func(value, tick_number):
        return f'{int(value)}'

# Assuming market_value_data is your DataFrame and 'Market Value' is the column
    #market_value_data['Market Value Numeric'].hist(bins=20)  # You can adjust the number of bins as needed

# Calculate the 80th and 90th percentiles
    percentile_95 = market_value_data['Market Value Numeric'].quantile(0.95)
    percentile_98 = market_value_data['Market Value Numeric'].quantile(0.98)
    percentile_99 = market_value_data['Market Value Numeric'].quantile(0.99)

# Setting up the X-axis and Y-axis ticks to display full numbers
    # ax = plt.gca()
    # ax.xaxis.set_major_formatter(ticker.FuncFormatter(format_func))
    # ax.yaxis.set_major_formatter(ticker.FuncFormatter(format_func))
    # ax.xaxis.set_major_locator(ticker.MaxNLocator(5))  # Adjust the number of ticks as needed
    # ax.set_yscale('log')  # Set the Y-axis to a logarithmic scale

# Add vertical lines for the 80th and 90th percentiles
    #ax.axvline(percentile_95, color='red', linestyle='dashed', linewidth=1, label=f'95th percentile ({int(percentile_95)})')
    #ax.axvline(percentile_99, color='blue', linestyle='dashed', linewidth=1, label=f'99th percentile ({int(percentile_99)})')

    #plt.title('Histogram of Market Value')
    #plt.xlabel('Market Value')
    #plt.ylabel('Frequency')
    #plt.legend()
    #plt.show()
    return percentile_95, percentile_98#, percentile_99

In [ ]:
market_value_data['Confederation'] = ''

In [ ]:
fifa_confederations = {
    'United States': 'CONCACAF',
    'Turkey': 'UEFA',
    'Republic of Ireland': 'UEFA',
    'Democratic Republic of the Congo': 'CAF',
    'Bosnia-Herzegovina': 'UEFA',
    'Republic of the Congo': 'CAF',
    'Solomon Islands': 'OFC',
    'New Caledonia': 'OFC',
    'Tahiti': 'OFC',
    'Papua New Guinea': 'OFC',
    'Vanuatu': 'OFC',
    'Cook Islands': 'OFC',
    'Samoa': 'OFC',
    'American Samoa': 'OFC',
    'Brunei Darussalam': 'AFC',
    'Tonga': 'OFC',
    'Turks and Caicos Islands': 'CONCACAF',
    'United States Virgin Islands': 'CONCACAF'
}

In [ ]:
not_found_set = []
for index, row in market_value_data.iterrows():
    country = row['Nationality']
    if country in latest_match_data['Team.2.Name'].unique():
        market_value_data.at[index, 'Confederation'] = latest_match_data[latest_match_data['Team.2.Name'] == country]['Team.2.Confederation'].unique()[0]
    else:
        confederation = fifa_confederations[f"{country}"]
        market_value_data.at[index, 'Confederation'] = confederation
        # if country in not_found_set:
        #     0==0
        # else:
        #     not_found_set.append(country)
        #     confederation = fifa_confederations[f"{country}"]
        #     market_value_data.at[index, 'Confederation'] = confederation

In [ ]:
market_value_data['Confederation'].value_counts()

In [ ]:
market_value_data.to_csv('market_value_data_confederations_added_june_18.csv')

In [ ]:
seasons_percentiles_DF_confederations = pd.DataFrame()
for confederation in market_value_data['Confederation'].unique():
    for season in market_value_data['Season'].unique():
        percentiles_values = plot_data_mv_confederation(market_value_data[(market_value_data['Season'] == season) & (market_value_data['Confederation'] == confederation)])
        new_row = {
            'Season': season,
            'Confederation': confederation,
            '95th Percentile, Value': percentiles_values[0],
            '98th Percentile, Value': percentiles_values[1],
        }
        new_row_df = pd.DataFrame([new_row])
        seasons_percentiles_DF_confederations = pd.concat([seasons_percentiles_DF_confederations, new_row_df], ignore_index=True)
seasons_percentiles_DF_confederations = seasons_percentiles_DF_confederations.sort_values('Season').reset_index().drop(columns={'index'})
seasons_percentiles_DF_confederations

In [ ]:
seasons_percentiles_DF = pd.DataFrame()
for season in market_value_data['Season'].unique():
    percentiles_values = plot_data_mv(market_value_data[market_value_data['Season'] == season])
    new_row = {
        'Season': season,
        '95th Percentile, Value': percentiles_values[0],
        '99th Percentile, Value': percentiles_values[1],
    }
    new_row_df = pd.DataFrame([new_row])
    seasons_percentiles_DF = pd.concat([seasons_percentiles_DF, new_row_df], ignore_index=True)
seasons_percentiles_DF = seasons_percentiles_DF.sort_values('Season').reset_index().drop(columns={'index'})
seasons_percentiles_DF

In [ ]:
for index, row in latest_match_data.iterrows():
    if row['Team.2.Name'] == 'Hong Kong':
        latest_match_data.at[index, 'Team.2.Confederation'] = 'AFC'
    if row['Team.1.Name'] == 'Hong Kong':
        latest_match_data.at[index, 'Team.1.Confederation'] = 'AFC'

In [ ]:
latest_match_data['Team 1, Tier 1 Global Superstars'] = 0
latest_match_data['Team 1, Tier 2 Global Superstars'] = 0
latest_match_data['Team 2, Tier 1 Global Superstars'] = 0
latest_match_data['Team 2, Tier 2 Global Superstars'] = 0

latest_match_data['Team 1, Tier 1 Confederation Stars'] = 0
latest_match_data['Team 1, Tier 2 Confederation Stars'] = 0
latest_match_data['Team 2, Tier 1 Confederation Stars'] = 0
latest_match_data['Team 2, Tier 2 Confederation Stars'] = 0

for index, row in latest_match_data.iterrows():
# for i in range(0, 1):
#     index = i 
#     row = latest_match_data.loc[index]
    team_1_value_set = eval(row['Team.1.Values.By.Player'])
    team_2_value_set = eval(row['Team.2.Values.By.Player'])
    team_1_confed = row['Team.1.Confederation']
    team_2_confed = row['Team.2.Confederation']

    if row['Season'] == 2024:
        season_num = 2023
    else:
        season_num = row['Season']

    row_season_global_tier_2_number = seasons_percentiles_DF[seasons_percentiles_DF['Season'] == season_num]['95th Percentile, Value'].unique()[0]
    row_season_global_tier_1_number = seasons_percentiles_DF[seasons_percentiles_DF['Season'] == season_num]['99th Percentile, Value'].unique()[0]

    row_season_confed_tier_2_number_team_2 = seasons_percentiles_DF_confederations[(seasons_percentiles_DF_confederations['Season'] == season_num) & (seasons_percentiles_DF_confederations['Confederation'] == team_2_confed)]['95th Percentile, Value'].unique()[0]
    row_season_confed_tier_1_number_team_2 = seasons_percentiles_DF_confederations[(seasons_percentiles_DF_confederations['Season'] == season_num) & (seasons_percentiles_DF_confederations['Confederation'] == team_2_confed)]['98th Percentile, Value'].unique()[0]

    row_season_confed_tier_2_number_team_1 = seasons_percentiles_DF_confederations[(seasons_percentiles_DF_confederations['Season'] == season_num) & (seasons_percentiles_DF_confederations['Confederation'] == team_1_confed)]['95th Percentile, Value'].unique()[0]
    row_season_confed_tier_1_number_team_1 = seasons_percentiles_DF_confederations[(seasons_percentiles_DF_confederations['Season'] == season_num) & (seasons_percentiles_DF_confederations['Confederation'] == team_1_confed)]['98th Percentile, Value'].unique()[0]

    team_1_true_count_tier_1_global_superstar = 0
    team_1_true_count_tier_2_global_superstar = 0
    team_1_true_count_tier_1_confed_superstar = 0
    team_1_true_count_tier_2_confed_superstar = 0
    team_2_true_count_tier_1_global_superstar = 0
    team_2_true_count_tier_2_global_superstar = 0
    team_2_true_count_tier_1_confed_superstar = 0
    team_2_true_count_tier_2_confed_superstar = 0

    for number_value in team_1_value_set:
        if number_value > row_season_global_tier_1_number:
            team_1_true_count_tier_1_global_superstar += 1
        elif number_value > row_season_global_tier_2_number:
            team_1_true_count_tier_2_global_superstar += 1
        if number_value > row_season_confed_tier_1_number_team_1:
            team_1_true_count_tier_1_confed_superstar += 1
        elif number_value > row_season_confed_tier_2_number_team_1:
            team_1_true_count_tier_2_confed_superstar += 1

    for number_value_2 in team_2_value_set:
        if number_value_2 > row_season_global_tier_1_number:
            team_2_true_count_tier_1_global_superstar += 1
        elif number_value_2 > row_season_global_tier_2_number:
            team_2_true_count_tier_2_global_superstar += 1
        if number_value_2 > row_season_confed_tier_1_number_team_1:
            team_2_true_count_tier_1_confed_superstar += 1
        elif number_value_2 > row_season_confed_tier_2_number_team_1:
            team_2_true_count_tier_2_confed_superstar += 1

    #print("Team 1, Number of Tier 1 Global superstars: ", team_1_true_count_tier_1_global_superstar)
    latest_match_data.at[index, 'Team 1, Tier 1 Global Superstars'] = team_1_true_count_tier_1_global_superstar
    #print("Team 1, Number of Tier 2 Global superstars: ", team_1_true_count_tier_2_global_superstar)
    latest_match_data.at[index, 'Team 1, Tier 2 Global Superstars'] = team_1_true_count_tier_2_global_superstar

    #print(f"Team 1, Number of Tier 1 {team_1_confed} stars: ", team_1_true_count_tier_1_confed_superstar)
    latest_match_data.at[index, 'Team 1, Tier 1 Confederation Stars'] = team_1_true_count_tier_1_confed_superstar
    #print(f"Team 1, Number of Tier 2 {team_1_confed} stars: ", team_1_true_count_tier_2_confed_superstar)
    latest_match_data.at[index, 'Team 1, Tier 2 Confederation Stars'] = team_1_true_count_tier_2_confed_superstar

    #print("Team 2, Number of Tier 1 Global superstars: ", team_2_true_count_tier_1_global_superstar)
    latest_match_data.at[index, 'Team 2, Tier 1 Global Superstars'] = team_2_true_count_tier_1_global_superstar
    #print("Team 2, Number of Tier 2 Global superstars: ", team_2_true_count_tier_2_global_superstar)
    latest_match_data.at[index, 'Team 2, Tier 2 Global Superstars'] = team_2_true_count_tier_2_global_superstar

    #print(f"Team 2, Number of Tier 1 {team_2_confed} stars: ", team_2_true_count_tier_1_confed_superstar)
    latest_match_data.at[index, 'Team 2, Tier 1 Confederation Stars'] = team_2_true_count_tier_1_confed_superstar
    #print(f"Team 2, Number of Tier 2 {team_2_confed} stars: ", team_2_true_count_tier_2_confed_superstar)
    latest_match_data.at[index, 'Team 2, Tier 2 Confederation Stars'] = team_2_true_count_tier_2_confed_superstar

#print(team_1_value_set, team_1_confed, season_num)
#print(row_season_global_tier_1_number, row_season_global_tier_2_number, row_season_confed_tier_1_number_team_1, row_season_confed_tier_2_number_team_1)

In [ ]:
latest_match_data.to_csv('latest_match_data_june_18_SUPERSTAR_INFO_ADDED.csv')

In [ ]:
latest_match_data.sort_values('Team 1, Tier 1 Global Superstars')